<a href="https://colab.research.google.com/github/ayyaz-azeem/NLP/blob/main/nlp_spam_classifier_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
PS = PorterStemmer()
import pandas as pd
messages = pd.read_csv('/content/SMSSpamCollection',sep = '\t',names = ["label","message"])


In [ ]:
pd.DataFrame(messages)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
from nltk.stem import WordNetLemmatizer
WN = WordNetLemmatizer()

In [ ]:
print(len(messages))

5572


In [ ]:
corpus_PS = []
corpus_WN = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['message'][i])
  review = review.lower()
  review = review.split()
  
  review_PS = [PS.stem(j) for j in review if j not in set(stopwords.words('english'))]
  review_WN = [WN.lemmatize(j) for j in review if j not in set(stopwords.words('english'))]
  review_PS = ' '.join(review_PS)
  review_WN = ' '.join(review_WN)
  corpus_PS.append(review_PS)
  corpus_WN.append(review_WN)

In [14]:

for i in range(0,5,1):
  if i>10:
    break
  print('\033[92m{}'.format(messages['message'][i]))
  print('\033[94m-{}'.format(corpus_PS[i]))
  print('\033[95m--{}'.format(corpus_WN[i]))
  print()

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
-go jurong point crazi avail bugi n great world la e buffet cine got amor wat
--go jurong point crazy available bugis n great world la e buffet cine got amore wat

Ok lar... Joking wif u oni...
-ok lar joke wif u oni
--ok lar joking wif u oni

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
-free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli
--free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply

U dun say so early hor... U c already then say...
-u dun say earli hor u c alreadi say
--u dun say early hor u c already say

Nah I don't think he goes to usf, he lives around here though
-nah think goe usf live around though
--nah think go usf life around though



In [ ]:
# BOW
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(max_features = 2500)
X_PS = CV.fit_transform(corpus_PS).toarray()
X_WN = CV.fit_transform(corpus_WN).toarray()

In [15]:
y = pd.get_dummies(messages['label'])
print('{}'.format(y))
y=y.iloc[:,1].values # we remove 1 column 
print('{}'.format(y))

      ham  spam
0       1     0
1       1     0
2       0     1
3       1     0
4       1     0
...   ...   ...
5567    0     1
5568    1     0
5569    1     0
5570    1     0
5571    1     0

[5572 rows x 2 columns]
[0 0 1 ... 0 0 0]


In [16]:
len(y)

5572

In [17]:
y.shape

(5572,)

In [18]:
from sklearn.model_selection import train_test_split
X_train_PS, X_test_PS, y_train_PS, y_test_PS = train_test_split(X_PS,y,test_size = 0.20, random_state = 0)

In [20]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model_PS = MultinomialNB().fit(X_train_PS, y_train_PS)

In [21]:
y_pred_PS = spam_detect_model_PS.predict(X_test_PS)

In [22]:
from sklearn.metrics import confusion_matrix
confusion_m_PS = confusion_matrix(y_test_PS,y_pred_PS)
print(confusion_m_PS)

[[946   9]
 [  7 153]]


In [25]:
from sklearn.metrics import accuracy_score
accuracy_PS = accuracy_score(y_test_PS,y_pred_PS)
print(accuracy_PS)

0.9856502242152466


In [24]:
X_train_WN, X_test_WN, y_train_WN, y_test_WN = train_test_split(X_WN,y,test_size = 0.20, random_state = 0)
spam_detect_model_WN = MultinomialNB().fit(X_train_WN, y_train_WN)
y_pred_WN = spam_detect_model_WN.predict(X_test_WN)
confusion_m_WN = confusion_matrix(y_test_WN,y_pred_WN)
print(confusion_m_WN)

[[946   9]
 [ 10 150]]


In [26]:
accuracy_WN = accuracy_score(y_test_WN,y_pred_WN)
print(accuracy_WN)

0.9829596412556054
